In [ ]:
df_match_merged.head()
print(df_match_merged['game_mode'].unique())
# [22  4  5  3  2 16  1]

match_id_game_mode_22 = df_match_merged[df_match_merged['game_mode']==22]['match_id'].iloc[0]
match_id_game_mode_04 = df_match_merged[df_match_merged['game_mode']==4]['match_id'].iloc[0]
match_id_game_mode_05 = df_match_merged[df_match_merged['game_mode']==5]['match_id'].iloc[0]
match_id_game_mode_03 = df_match_merged[df_match_merged['game_mode']==3]['match_id'].iloc[0]
match_id_game_mode_02 = df_match_merged[df_match_merged['game_mode']==2]['match_id'].iloc[0]
match_id_game_mode_16 = df_match_merged[df_match_merged['game_mode']==16]['match_id'].iloc[0]
match_id_game_mode_01 = df_match_merged[df_match_merged['game_mode']==1]['match_id'].iloc[0]

print(match_id_game_mode_01) # All pick
print(match_id_game_mode_02) # Captains mode
print(match_id_game_mode_03) # Random Draft
print(match_id_game_mode_04) # Single Draft
print(match_id_game_mode_05) # All random
print(match_id_game_mode_16) # Captains draft
print(match_id_game_mode_22) # All draft

print(df_match_merged['lobby_type'].unique())

[22  4  5  3  2 16  1]
6249738002
6246238706
6246231305
6246229807
6246230205
6247583407
6246229802
[7 0 1 9]
